query = """
    SELECT g.gu_name, d.dong_name, count
    FROM gu g, dong d, hospital h
    WHERE (g.gu_id = d.gu_id AND d.dong_id = h.dong_id)
    AND h.hospital_name LIKE '%소아%'
    GROUP BY d.dong_name
"""


df = pd.read_sql(query, conn)

df